# Building and analysing a small community model

Here we will use `COBREXA` to build and analyze a small community model
consisting of three *E. coli* mutants using the `CoreModel`. We will use an
objective function that enforces equal growth rates.

We will first construct a community of only two mutants to illustrate the
effect of the community biomass objective function. Then we will add a third
member that has a lethal knockout. Due to the bounds on the exchange reactions
these three models are incapable of sharing resources - hence the maximum
growth rate will be zero. By changing the bounds we can allow resource sharing,
"saving" the community.

## Load the base model

In [1]:
!isfile("e_coli_core.json") &&
    download("http://bigg.ucsd.edu/static/models/e_coli_core.json", "e_coli_core.json");

using COBREXA
using Tulip

## Load the models and inspect fba solutions

In [2]:
base_model = load_model(CoreModel, "e_coli_core.json") # base from from which the knockouts will be constructed

cytbd_knockout_model = remove_reactions(base_model, "CYTBD") # knockout the CYTBD (cytochrome oxidase) reaction
sol = flux_balance_analysis_dict(cytbd_knockout_model, Tulip.Optimizer)
sol["BIOMASS_Ecoli_core_w_GAM"] # Cytochrome oxidase knockout μ (growth rate)

0.21166294865467686

In [3]:
atps4r_knockout_model = remove_reactions(base_model, "ATPS4r") # knockout the ATP synthase reaction
sol = flux_balance_analysis_dict(atps4r_knockout_model, Tulip.Optimizer)
sol["BIOMASS_Ecoli_core_w_GAM"] # ATP synthase knockout μ

0.37422987390664064

In [4]:
eno_knockout_model = remove_reactions(base_model, "ENO") # knockout the enolase reaction
sol = flux_balance_analysis_dict(eno_knockout_model, Tulip.Optimizer)
sol["BIOMASS_Ecoli_core_w_GAM"] # Enolase knockout μ, cannot grow by itself

9.548564285305505e-16

## Build a community model of the cytochrome oxidase knockout and the ATP synthase knockout models

In [5]:
ex_rxns = filter(looks_like_exchange_reaction, reactions(base_model)) # identify exchange reactions heuristically
ex_mets = [first(keys(reaction_stoichiometry(base_model, ex_rxn))) for ex_rxn in ex_rxns] # identify exchange metabolites IN THE SAME ORDER as ex_rxns
[ex_rxns ex_mets]

20×2 Matrix{String}:
 "EX_ac_e"      "ac_e"
 "EX_acald_e"   "acald_e"
 "EX_akg_e"     "akg_e"
 "EX_co2_e"     "co2_e"
 "EX_etoh_e"    "etoh_e"
 "EX_for_e"     "for_e"
 "EX_fru_e"     "fru_e"
 "EX_fum_e"     "fum_e"
 "EX_glc__D_e"  "glc__D_e"
 "EX_gln__L_e"  "gln__L_e"
 "EX_glu__L_e"  "glu__L_e"
 "EX_h_e"       "h_e"
 "EX_h2o_e"     "h2o_e"
 "EX_lac__D_e"  "lac__D_e"
 "EX_mal__L_e"  "mal__L_e"
 "EX_nh4_e"     "nh4_e"
 "EX_o2_e"      "o2_e"
 "EX_pi_e"      "pi_e"
 "EX_pyr_e"     "pyr_e"
 "EX_succ_e"    "succ_e"

In [6]:
model_names = ["cytbd_ko", "atps4r_ko"]
community_model = join_with_exchanges(
    [cytbd_knockout_model, atps4r_knockout_model],
    ex_rxns,
    ex_mets;
    add_biomass_objective = true,
    biomass_ids = ["BIOMASS_Ecoli_core_w_GAM", "BIOMASS_Ecoli_core_w_GAM"],
    model_names = model_names,
)

Metabolic model of type CoreModel

⡀⣄⢠⣤⣼⢠⣄⡀⠀⠘⣄⠀⡄⣌⣿⡦⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠤⠈⠀⢤⠀⠃⡄⠀⠀⠐⠀⠀⢢⣤⣬⢷⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠘⣐⢠⣤⣰⠠⠆⡀⠀⠀⠀⠃⡠⢨⡀⡀⠠⠘⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠁⣈⡁⡉⠸⠤⠰⣠⡀⠀⠀⠑⠸⠈⠀⠄⠉⠎⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣁⡁⡥⣀⣏⠜⠐⡢⠁⠀⠀⠐⠐⠀⣀⢐⠁⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠍⢅⠑⠋⢯⠲⢀⣅⢜⠀⠀⠀⠄⠀⠁⠒⠔⠰⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⢉⠀⠀⠀⢨⠀⠠⠭⠄⠢⠀⠀⠿⢤⠀⠀⠀⠀⠀⠀⠀⠀⡀⠀⠀⠀⠀⢀⠀⠀⠀⢀⣀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠴⠤⢶⠴⠧⢶⠦⠄⠀⠘⠄⠀⠆⠮⠿⣶⠆⠄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⠒⠀⠐⠆⢈⠀⠆⠀⠀⢈⠀⠀⢷⠶⠖⠻⣖⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡨⢡⡦⣲⣇⠂⠣⠀⠀⠀⠀⡁⢔⢴⠄⢄⢂⢈⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⣤⠀⠄⡊⡚⠘⠴⡄⠀⠀⠁⠘⠀⠀⠂⡀⢃⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣤⡄⣶⣬⡆⠊⢩⠐⡀⠀⠀⠈⠈⢠⡤⠌⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡂⠢⠉⠙⡎⠃⠰⣴⠨⠀⠀⢐⣀⠀⠀⠁⠊⠘⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠄⠀⠀⠀⠆⠀⠐⠒⣄⠃⠀⠐⠙⠶⠀⠀⠀⠠⣄⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠈⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀
⠀⠀⠀⠀⠠⠀⠀⠀⠀⠀⠀⠑⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠄⠀⠀⠀⠀⠀⠈⠓⠀⠀⠀⠀⠀⠀⠀⠀⠈⠓
Number of reactions: 209
Number of metabolites: 166


## Set exchange reaction bounds of community model based on the bounds of the individual models

In [7]:
env_ex_rxn_idxs = indexin(ex_rxns, reactions(community_model)) # identify the global (environmental exchange reactions)
cytbd_ex_rxn_idxs = indexin(ex_rxns, reactions(cytbd_knockout_model)) # identify the indices of the corresponding exchange reactions in the original models
atps4r_ex_rxn_idxs = indexin(ex_rxns, reactions(atps4r_knockout_model))

20-element Vector{Union{Nothing, Int64}}:
 43
 44
 45
 46
 47
 48
 49
 50
 51
 52
 53
 54
 55
 56
 57
 58
 59
 60
 61
 62

In case some exchange reactions are not present in both models, set
environmental exchange bound to the sum of the individual exchange bounds

In [8]:
for (env_ex, m2_ex, m1_ex) in zip(env_ex_rxn_idxs, cytbd_ex_rxn_idxs, atps4r_ex_rxn_idxs)
    m2lb = isnothing(m2_ex) ? 0.0 : atps4r_knockout_model.xl[m2_ex]
    m2ub = isnothing(m2_ex) ? 0.0 : atps4r_knockout_model.xu[m2_ex]
    m1lb = isnothing(m1_ex) ? 0.0 : cytbd_knockout_model.xl[m1_ex]
    m1ub = isnothing(m1_ex) ? 0.0 : cytbd_knockout_model.xu[m1_ex]
    change_bounds!(community_model, [env_ex]; xl = [m1lb + m2lb], xu = [m1ub + m2ub])
end

## Add objective function to community model`

In [9]:
biomass_ids = model_names .* "_BIOMASS_Ecoli_core_w_GAM"
add_objective!(
    community_model,
    biomass_ids;
    objective_column_index = first(
        indexin(["community_biomass"], reactions(community_model)),
    ),
)

## Perform community FBA

In [10]:
d = flux_balance_analysis_dict(
    community_model,
    Tulip.Optimizer;
    modifications = [change_optimizer_attribute("IPM_IterationsLimit", 1000)],
)
d["community_biomass"] # community μ

0.21166294961691223

Notice, the growth rate is limited to the slowest organism as per the objective function

## Add the enolase knockout to the community model

In [11]:
community_model = add_model_with_exchanges(
    community_model,
    eno_knockout_model,
    ex_rxns,
    ex_mets;
    model_name = "eno_ko",
    biomass_id = "BIOMASS_Ecoli_core_w_GAM",
)

push!(model_names, "eno_ko")
biomass_ids = model_names .* "_BIOMASS_Ecoli_core_w_GAM"
add_objective!(
    community_model,
    biomass_ids;
    objective_column_index = first(
        indexin(["community_biomass"], reactions(community_model)),
    ),
)

d = flux_balance_analysis_dict(
    community_model,
    Tulip.Optimizer;
    modifications = [change_optimizer_attribute("IPM_IterationsLimit", 1000)],
)
println("Community μ = ", d["community_biomass"])

Community μ = 1.6222069161577525e-15


Notice that the high communal growth rate is 0, due to the enolase knockout.
The reason for this behaviour: enolase is a central reaction in glycolysis - without
it the organism cannot access the lower glycolysis pathways or the TCA cycle, hence
the model predicts no growth for the knockout, and hence no growth for the system since
they all have to have the same growth rate.

## Allow the mutants to rescue each other by sharing pyruvate

In [12]:
pyr_exs = model_names .* "_EX_pyr_e"
change_bounds!(community_model, pyr_exs; xl = fill(-1000.0, 3), xu = fill(1000.0, 3))

d = flux_balance_analysis_dict(
    community_model,
    Tulip.Optimizer;
    modifications = [change_optimizer_attribute("IPM_IterationsLimit", 1000)],
)
d["community_biomass"] # community μ

0.23607110796208797

Notice that the growth rate is now above 0! Nutrient sharing saved the day!

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*